In [1]:
import pandas as pd
import numpy as npy
from datetime import datetime
from datetime import date
import matplotlib.pyplot as plt
import random
import sklearn
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
model=LogisticRegression()
import itertools

In [ ]:
#this cell contains all functions defined for building columns in ufcfightscrap
#converts from '%B %d, %Y' (i.e. August 22, 2020) to date (i.e. 2020-08-22)
def convert_to_datetime(day1):
    return datetime.strptime(day1, '%B %d, %Y').date()
convert_to_datetime_vect= npy.vectorize(convert_to_datetime)

#this age function is written in such a stupid way
def age(birthDate,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    if birthDate=='--':
        aa='unknown'
    elif type(birthDate)==str and not type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif type(birthDate)==str and type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif not type(birthDate)==str and type(day)==str:
        bd=birthDate
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    else:
        bd=birthDate
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    return aa

age_vect= npy.vectorize(age)

def fighter_age(fighter,day=date.today(),form1='%B %d, %Y',form2='%B %d, %Y'):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            dob=datetime.strptime(ufcfighterscrap['dob'][i], '%b %d, %Y').strftime('%B %d, %Y')
            a=age(dob,day,form1,form2)
            break
    return a

def wins_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
    return summ

def losses_before(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
    return summ

#functions for height and reach
def fighter_height(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['height'][i]
            break
    if a=='--':
        b='unknown'
    elif a[4]=='"':
        b=int(a[0])*30.48+int(a[3])*2.54
    else:
        b=int(a[0])*30.48+int(a[3]+a[4])*2.54
    return b

def fighter_reach(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['reach'][i]
            break
    if a=='--':
        b='unknown'
    else:
        b=int(a[0]+a[1])*2.54
    return b

wins_before_vect= npy.vectorize(wins_before)
losses_before_vect= npy.vectorize(losses_before)
fighter_height_vect= npy.vectorize(fighter_height)
fighter_reach_vect= npy.vectorize(fighter_reach)

#day1 should be input in the form '%B %d, %Y' i.e. 'August 20, 1962'
#conversions can be made via day=datetime.strptime(ufcfightsML_known_df['date'][i], '%B %d, %Y').strftime('%b %d, %Y')
def time_diff(day1,day2=date.today()):
    if day2==date.today():
        answer=(day2-datetime.strptime(day, '%B %d, %Y')).days
    else:
        answer=(datetime.strptime(day2, '%B %d, %Y')-datetime.strptime(day1, '%B %d, %Y')).days
    return answer

#we now vectorize this to use in pandas/numpy
time_diff_vect= npy.vectorize(time_diff)

#can make a single function to do all of these... actually maybe the count function would even work as is

def L5Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L5Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L2Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def L2Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return summ

def ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return summ

def L5Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L5Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L2Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def L2Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return summ

def sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
    return summ

def L5Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L5Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<1825 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>1825:
            break
    return summ

def L2Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='W' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

def L2Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    for i in good_indices:
        if 0<time_diff(ufcfightscrap['date'][i],day1)<730 and ufcfightscrap['result'][i]=='L' and ufcfightscrap['method'][i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date'][i],day1)>730:
            break
    return summ

#vectorize all of these functions
L5Y_wins_vect= npy.vectorize(L5Y_wins)
L5Y_losses_vect= npy.vectorize(L5Y_losses)
L2Y_wins_vect= npy.vectorize(L2Y_wins)
L2Y_losses_vect= npy.vectorize(L2Y_losses)
ko_wins_vect= npy.vectorize(ko_wins)
ko_losses_vect= npy.vectorize(ko_losses)
L5Y_ko_wins_vect= npy.vectorize(L5Y_ko_wins)
L5Y_ko_losses_vect= npy.vectorize(L5Y_ko_losses)
L2Y_ko_wins_vect= npy.vectorize(L2Y_ko_wins)
L2Y_ko_losses_vect= npy.vectorize(L2Y_ko_losses)
sub_wins_vect= npy.vectorize(sub_wins)
sub_losses_vect= npy.vectorize(sub_losses)
L5Y_sub_wins_vect= npy.vectorize(L5Y_sub_wins)
L5Y_sub_losses_vect= npy.vectorize(L5Y_sub_losses)
L2Y_sub_wins_vect= npy.vectorize(L2Y_sub_wins)
L2Y_sub_losses_vect= npy.vectorize(L2Y_sub_losses)

#for columns like fighter_rec which contains the information for the opponent as well, we use the following
def opponent_column(stat):
    col=dict()
    for i in range(len(ufcfightscrap['fighter'])):
        if i%2==0:
            col[i]=ufcfightscrap[stat][i+1]
        else:
            col[i]=ufcfightscrap[stat][i-1]
    statdict={'stat':col}
    return pd.DataFrame (statdict, columns = ['stat'])

#enter date unabbreviated 'July 4, 2019'
#here the average gives avg per fight. Later in avg_count we change to average per time spent in octagon
def count(stat, guy,inf_abs, total_L5Y_L2Y_avg, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if total_L5Y_L2Y_avg=='total' or total_L5Y_L2Y_avg=='avg':
        good_indices_1=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0]
    elif total_L5Y_L2Y_avg=='L2Y':
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<730]
    else:
        good_indices_1=[i for i in ufcfightscrap.index.values if 0<time_diff(ufcfightscrap['date'][i],day1)<1825]
    if inf_abs=='inf':
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy]
    else:
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy]
    good_indices=[i for i in good_indices_1 if i in good_indices_2]
    if total_L5Y_L2Y_avg!='avg':
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
    else:
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
        day1=convert_date_to_abbrev(day1)
        number_fights=wins_before(guy,day1)+losses_before(guy,day1)
        summ=summ/float(number_fights)
    return summ
        
#note a better average is per time not per fight. We will do each stat as an average per one minutes

#enter time in the form 'August 24, 2018'
def time_in_octagon(guy,day1=date.today().strftime('%B %d, %Y')):
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date'][i],day1)>0 and ufcfightscrap['fighter'][i]==guy] 
    for i in good_indices:
        if ufcfightscrap['time'][i][2]==':':
            summ=int(ufcfightscrap['time'][i][0:2])+int(ufcfightscrap['time'][i][3:])/60.0
        else:
            summ+=5*(ufcfightscrap['round'][i]-1)+int(ufcfightscrap['time'][i][0])+int(ufcfightscrap['time'][i][2:])/60.0
    return summ

#enter date unabbreviated 'July 4, 2019'
#gives takedowns per minute
def avg_count(stat, guy,inf_abs, day1=date.today().strftime('%B %d, %Y')):
    summ=0
    if inf_abs=='inf':
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    else:
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'][i]==guy and time_diff(ufcfightscrap['date'][i],day1)>0]
    for i in good_indices:
        summ+=ufcfightscrap[stat][i]
    t= time_in_octagon(guy,day1)
    if t==0:
        summ=0
    else:
        summ=summ/t
    return summ

#vectorize these functions
count_vect= npy.vectorize(count)
avg_count_vect= npy.vectorize(avg_count)

#need to define a model first before calling the following functions
#REDEFINE THIS AFTER GETTING ALL STATS COMPILED. WE CURRENTLY CANNOT PREDICT
#I've redefined this in such a way to predict what happens when fighter1 in their day1 version fights fighter2
#in their day2 version. Meaning we could compare for example 2014 Tyron Woodley to 2019 Colby Covington
def ufc_prediction_tuple(fighter1,fighter2,day1=date.today(),day2=date.today()):
    return [wins_before(fighter1,day1)-wins_before(fighter2,day2),
            losses_before(fighter1,day1)-losses_before(fighter2,day2),
            L5Y_wins(fighter1,day1)-L5Y_wins(fighter2,day2),
            L5Y_losses(fighter1,day1)-L5Y_losses(fighter2,day2),
            L2Y_wins(fighter1,day1)-L2Y_wins(fighter2,day2),
            L2Y_losses(fighter1,day1)-L2Y_losses(fighter2,day2),
            ko_wins(fighter1,day1)-ko_wins(fighter2,day2),
            ko_losses(fighter1,day1)-ko_losses(fighter2,day2),
            L5Y_ko_wins(fighter1,day1)-L5Y_ko_wins(fighter2,day2),
            L5Y_ko_losses(fighter1,day1)-L5Y_ko_losses(fighter2,day2),
            L2Y_ko_wins(fighter1,day1)-L2Y_ko_wins(fighter2,day2),
            L2Y_ko_losses(fighter1,day1)-L2Y_ko_losses(fighter2,day2),
            sub_wins(fighter1,day1)-sub_wins(fighter2,day2),
            sub_losses(fighter1,day1)-sub_losses(fighter2,day2),
            L5Y_sub_wins(fighter1,day1)-L5Y_sub_wins(fighter2,day2),
            L5Y_sub_losses(fighter1,day1)-L5Y_sub_losses(fighter2,day2),
            L2Y_sub_wins(fighter1,day1)-L2Y_sub_wins(fighter2,day2),
            L2Y_sub_losses(fighter1,day1)-L2Y_sub_losses(fighter2,day2),
            fighter_age(fighter1,day1),fighter_age(fighter2,day2),
            fighter_height(fighter1)-fighter_height(fighter2),
            fighter_reach(fighter1)-fighter_reach(fighter2),
            avg_count('knockdowns',fighter1,'inf',day1), avg_count('pass',fighter1,'inf',day1),
            avg_count('reversals',fighter1,'inf',day1),avg_count('sub_attempts',fighter1,'inf',day1),
            avg_count('takedowns_landed',fighter1,'inf',day1),avg_count('takedowns_attempts',fighter1,'inf',day1),
            avg_count('sig_strikes_landed',fighter1,'inf',day1),avg_count('sig_strikes_attempts',fighter1,'inf',day1),
            avg_count('total_strikes_landed',fighter1,'inf',day1),avg_count('total_strikes_attempts',fighter1,'inf',day1),
            avg_count('head_strikes_landed',fighter1,'inf',day1),avg_count('head_strikes_attempts',fighter1,'inf',day1),
            avg_count('body_strikes_landed',fighter1,'inf',day1),avg_count('body_strikes_attempts',fighter1,'inf',day1),
            avg_count('leg_strikes_landed',fighter1,'inf',day1),avg_count('leg_strikes_attempts',fighter1,'inf',day1),
            avg_count('distance_strikes_landed',fighter1,'inf',day1),avg_count('distance_strikes_attempts',fighter1,'inf',day1),
            avg_count('clinch_strikes_landed',fighter1,'inf',day1),avg_count('clinch_strikes_attempts',fighter1,'inf',day1),
            avg_count('ground_strikes_landed',fighter1,'inf',day1),avg_count('ground_strikes_attempts',fighter1,'inf',day1),
            avg_count('knockdowns',fighter1,'abs',day1), avg_count('pass',fighter1,'abs',day1),
            avg_count('reversals',fighter1,'abs',day1),avg_count('sub_attempts',fighter1,'abs',day1),
            avg_count('takedowns_landed',fighter1,'abs',day1),avg_count('takedowns_attempts',fighter1,'abs',day1),
            avg_count('sig_strikes_landed',fighter1,'abs',day1),avg_count('sig_strikes_attempts',fighter1,'abs',day1),
            avg_count('total_strikes_landed',fighter1,'abs',day1),avg_count('total_strikes_attempts',fighter1,'abs',day1),
            avg_count('head_strikes_landed',fighter1,'abs',day1),avg_count('head_strikes_attempts',fighter1,'abs',day1),
            avg_count('body_strikes_landed',fighter1,'abs',day1),avg_count('body_strikes_attempts',fighter1,'abs',day1),
            avg_count('leg_strikes_landed',fighter1,'abs',day1),avg_count('leg_strikes_attempts',fighter1,'abs',day1),
            avg_count('distance_strikes_landed',fighter1,'abs',day1),avg_count('distance_strikes_attempts',fighter1,'abs',day1),
            avg_count('clinch_strikes_landed',fighter1,'abs',day1),avg_count('clinch_strikes_attempts',fighter1,'abs',day1),
            avg_count('ground_strikes_landed',fighter1,'abs',day1),avg_count('ground_strikes_attempts',fighter1,'abs',day1),
            avg_count('knockdowns',fighter2,'inf',day2), avg_count('pass',fighter2,'inf',day2),
            avg_count('reversals',fighter2,'inf',day2),avg_count('sub_attempts',fighter2,'inf',day2),
            avg_count('takedowns_landed',fighter2,'inf',day2),avg_count('takedowns_attempts',fighter2,'inf',day2),
            avg_count('sig_strikes_landed',fighter2,'inf',day2),avg_count('sig_strikes_attempts',fighter2,'inf',day2),
            avg_count('total_strikes_landed',fighter2,'inf',day2),avg_count('total_strikes_attempts',fighter2,'inf',day2),
            avg_count('head_strikes_landed',fighter2,'inf',day2),avg_count('head_strikes_attempts',fighter2,'inf',day2),
            avg_count('body_strikes_landed',fighter2,'inf',day2),avg_count('body_strikes_attempts',fighter2,'inf',day2),
            avg_count('leg_strikes_landed',fighter2,'inf',day2),avg_count('leg_strikes_attempts',fighter2,'inf',day2),
            avg_count('distance_strikes_landed',fighter2,'inf',day2),avg_count('distance_strikes_attempts',fighter2,'inf',day2),
            avg_count('clinch_strikes_landed',fighter2,'inf',day2),avg_count('clinch_strikes_attempts',fighter2,'inf',day2),
            avg_count('ground_strikes_landed',fighter2,'inf',day2),avg_count('ground_strikes_attempts',fighter2,'inf',day2),
            avg_count('knockdowns',fighter2,'abs',day2), avg_count('pass',fighter2,'abs',day2),
            avg_count('reversals',fighter2,'abs',day2),avg_count('sub_attempts',fighter2,'abs',day2),
            avg_count('takedowns_landed',fighter2,'abs',day2),avg_count('takedowns_attempts',fighter2,'abs',day2),
            avg_count('sig_strikes_landed',fighter2,'abs',day2),avg_count('sig_strikes_attempts',fighter2,'abs',day2),
            avg_count('total_strikes_landed',fighter2,'abs',day2),avg_count('total_strikes_attempts',fighter2,'abs',day2),
            avg_count('head_strikes_landed',fighter2,'abs',day2),avg_count('head_strikes_attempts',fighter2,'abs',day2),
            avg_count('body_strikes_landed',fighter2,'abs',day2),avg_count('body_strikes_attempts',fighter2,'abs',day2),
            avg_count('leg_strikes_landed',fighter2,'abs',day2),avg_count('leg_strikes_attempts',fighter2,'abs',day2),
            avg_count('distance_strikes_landed',fighter2,'abs',day2),avg_count('distance_strikes_attempts',fighter2,'abs',day2),
            avg_count('clinch_strikes_landed',fighter2,'abs',day2),avg_count('clinch_strikes_attempts',fighter2,'abs',day2),
            avg_count('ground_strikes_landed',fighter2,'abs',day2),avg_count('ground_strikes_attempts',fighter2,'abs',day2),
            stance(fighter1),stance(fighter2)]


#date entered in the form 'Aug 20, 2013'
#predicts what happens if the day1 version of fighter1 fights fighter2 at their day2 version
#by default day1 and day2 are set to today if left unspecified
def ufc_predict(fighter1,fighter2,day1=date.today(), day2=date.today()):
    fighters=[fighter1,fighter2]
    random_guy=fighters[random.choice([0,1])]
    fighter1_fights=(ufcfightscrap['fighter']==fighter1).sum()
    fighter2_fights=(ufcfightscrap['fighter']==fighter2).sum()
    if fighter1_fights==0 and not fighter2_fights==0:
        print('Either you spelled something wrong, or this is '+fighter1+"'s UFC debut... so obviously I do not have enough data to make an educated pick. But since you are DEMANDING an outcome, hmmm... I guess I'll go with "+random_guy+'.')
    elif fighter2_fights==0 and not fighter1_fights==0:
        print('Either you spelled something wrong, or this is '+fighter2+"'s UFC debut... so obviously I do not have enough data to make an educated pick. But since you are DEMANDING an outcome, hmmm... I guess I'll go with "+random_guy+'.') 
    elif fighter1_fights==0 and fighter2_fights==0:
        print('Either you spelled some things wrong, or this is both '+fighter1+"'s "+'and '+fighter2+"'s "+"UFC debuts... so obviously I do not have enough data to make an educated pick. But since you are DEMANDING an outcome, hmmm... I guess I'll go with "+random_guy+'.')
    else:
        ufc_tuple=ufc_prediction_tuple(fighter1,fighter2,day1,day2)
        if 'unknown' in ufc_tuple or 'NaN' in ufc_tuple:
            print('Fighters have incomplete data on record, I honestly do not know what to do with this.')
        else:
            value=model.predict([ufc_tuple])[0]
            result=mlp.predict([ufc_tuple])[0]
            if value=='W':
                print(fighter1+' wins by '+result)
            else:
                print(fighter2+' wins by '+result)

def stance(fighter):
    a=0
    for i in range(len(ufcfighterscrap['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['stance'][i]
            break
    if a=='Orthodox':
        return 0
    elif a=='Switch':
        return 1
    elif a=='Southpaw':
        return 2
    elif a=='Open Stance':
        return 3
    elif a=='Sideways':
        return 4
    else:
        return 5
    
stance_vect= npy.vectorize(stance)

In [5]:
ufc_fights= pd.read_csv('ufc_fights.csv')
ufc_fights_crap= pd.read_csv('ufc_fights_crap.csv')
ufcfightscrap= pd.read_csv('ufc_fights_crap.csv')
ufcfighterscrap=pd.read_csv('fighter_stats.csv',sep=',')

/opt/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (36,37,101,102) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
#adding stance (Sorry, you'll have to wait a minute or so for this to evaluate. I should really have done this
# in the building_ufc_fights file, but I didn't... I'll fix this eventually)
#Not sure I did this in the best way, for instance i could change the column to 'categorical values'
#however this may be equivalent to what I did... not sure
ufc_fights['fighter_stance']=stance_vect(ufc_fights['fighter'])
ufc_fights['opponent_stance']=stance_vect(ufc_fights['opponent'])
stance_mask_a=ufc_fights['fighter_stance']!=5
stance_mask_b=ufc_fights['opponent_stance']!=5
ufc_fights_stance=ufc_fights[stance_mask_a&stance_mask_b]

#gets rid of all unknown, NaN, and any columns with UFC debut in ufc_fights_ML_dirty and outputs ufc_fights_ML
mask=ufc_fights_stance['fighter_wins'] != 'unknown'

for col in ufc_fights_stance.columns:
    mask1=ufc_fights_stance[col] != 'unknown'
    mask2=ufc_fights_stance[col] != 'NaN'
    mask3=ufc_fights_stance[col] != '--'
    mask=mask&mask1&mask2&mask3
    
maska=ufc_fights_stance['fighter_inf_sig_strikes_attempts_avg'] != 0
maskb=ufc_fights_stance['opponent_inf_sig_strikes_attempts_avg'] != 0
mask=mask&maska&maskb
ufc_fights_clean_stance=ufc_fights_stance[mask]
results=ufc_fights_clean_stance['result']
ufc_fights_ML=ufc_fights_clean_stance.drop(['result','fighter','opponent','method','date','division'], axis=1)

record_statistics=[u'fighter_wins', 
                   u'fighter_losses',
                   u'fighter_L5Y_wins',
                   u'fighter_L5Y_losses', 
                   u'fighter_L2Y_wins', 
                   u'fighter_L2Y_losses',
                    u'fighter_ko_wins',
                   u'fighter_ko_losses',
                   u'fighter_L5Y_ko_wins',
                   u'fighter_L5Y_ko_losses',
                   u'fighter_L2Y_ko_wins',
                    u'fighter_L2Y_ko_losses',
                   u'fighter_sub_wins',
                   u'fighter_sub_losses',
                   u'fighter_L5Y_sub_wins',
                    u'fighter_L5Y_sub_losses', 
                   u'fighter_L2Y_sub_wins', 
                   u'fighter_L2Y_sub_losses',
                   u'opponent_wins', 
                   u'opponent_losses',
                   u'opponent_L5Y_wins', 
                   u'opponent_L5Y_losses', 
                   u'opponent_L2Y_wins', 
                   u'opponent_L2Y_losses', 
                    u'opponent_ko_wins', 
                   u'opponent_ko_losses', 
                   u'opponent_L5Y_ko_wins', 
                   u'opponent_L5Y_ko_losses', 
                   u'opponent_L2Y_ko_wins',
                    u'opponent_L2Y_ko_losses', 
                   u'opponent_sub_wins', 
                   u'opponent_sub_losses',
                   u'opponent_L5Y_sub_wins', 
                    u'opponent_L5Y_sub_losses', 
                   u'opponent_L2Y_sub_wins', 
                   u'opponent_L2Y_sub_losses']

physical_stats=[ u'fighter_age',
                u'fighter_height',
                    u'fighter_reach',  
                u'opponent_age',  
                u'opponent_height',
                    u'opponent_reach']

#here is the list of all stats available (besides stance), does not include names or result
punch_statistics=[    u'fighter_inf_knockdowns_avg',
                    u'fighter_inf_pass_avg',
                    u'fighter_inf_reversals_avg',
                    u'fighter_inf_sub_attempts_avg',
                    u'fighter_inf_takedowns_landed_avg',
                    u'fighter_inf_takedowns_attempts_avg',
                    u'fighter_inf_sig_strikes_landed_avg',
                    u'fighter_inf_sig_strikes_attempts_avg',
                    u'fighter_inf_total_strikes_landed_avg',
                    u'fighter_inf_total_strikes_attempts_avg',
                    u'fighter_inf_head_strikes_landed_avg',
                    u'fighter_inf_head_strikes_attempts_avg',
                    u'fighter_inf_body_strikes_landed_avg',
                    u'fighter_inf_body_strikes_attempts_avg',
                    u'fighter_inf_leg_strikes_landed_avg',
                    u'fighter_inf_leg_strikes_attempts_avg',
                    u'fighter_inf_distance_strikes_landed_avg',
                    u'fighter_inf_distance_strikes_attempts_avg',
                    u'fighter_inf_clinch_strikes_landed_avg',
                    u'fighter_inf_clinch_strikes_attempts_avg',
                    u'fighter_inf_ground_strikes_landed_avg',
                    u'fighter_inf_ground_strikes_attempts_avg',
                
                    u'fighter_abs_knockdowns_avg',
                    u'fighter_abs_pass_avg',
                    u'fighter_abs_reversals_avg',
                    u'fighter_abs_sub_attempts_avg',
                    u'fighter_abs_takedowns_landed_avg',
                    u'fighter_abs_takedowns_attempts_avg',
                    u'fighter_abs_sig_strikes_landed_avg',
                    u'fighter_abs_sig_strikes_attempts_avg',
                    u'fighter_abs_total_strikes_landed_avg',
                    u'fighter_abs_total_strikes_attempts_avg',
                    u'fighter_abs_head_strikes_landed_avg',
                    u'fighter_abs_head_strikes_attempts_avg',
                    u'fighter_abs_body_strikes_landed_avg',
                    u'fighter_abs_body_strikes_attempts_avg',
                    u'fighter_abs_leg_strikes_landed_avg',
                    u'fighter_abs_leg_strikes_attempts_avg',
                    u'fighter_abs_distance_strikes_landed_avg',
                    u'fighter_abs_distance_strikes_attempts_avg',
                    u'fighter_abs_clinch_strikes_landed_avg',
                    u'fighter_abs_clinch_strikes_attempts_avg',
                    u'fighter_abs_ground_strikes_landed_avg',
                    u'fighter_abs_ground_strikes_attempts_avg',
                    
                    u'opponent_inf_knockdowns_avg',
                    u'opponent_inf_pass_avg',
                    u'opponent_inf_reversals_avg',
                    u'opponent_inf_sub_attempts_avg',
                    u'opponent_inf_takedowns_landed_avg',
                    u'opponent_inf_takedowns_attempts_avg',
                    u'opponent_inf_sig_strikes_landed_avg',
                    u'opponent_inf_sig_strikes_attempts_avg',
                    u'opponent_inf_total_strikes_landed_avg',
                    u'opponent_inf_total_strikes_attempts_avg',
                    u'opponent_inf_head_strikes_landed_avg',
                    u'opponent_inf_head_strikes_attempts_avg',
                    u'opponent_inf_body_strikes_landed_avg',
                    u'opponent_inf_body_strikes_attempts_avg',
                    u'opponent_inf_leg_strikes_landed_avg',
                    u'opponent_inf_leg_strikes_attempts_avg',
                    u'opponent_inf_distance_strikes_landed_avg',
                    u'opponent_inf_distance_strikes_attempts_avg',
                    u'opponent_inf_clinch_strikes_landed_avg',
                    u'opponent_inf_clinch_strikes_attempts_avg',
                    u'opponent_inf_ground_strikes_landed_avg',
                    u'opponent_inf_ground_strikes_attempts_avg',
                
                    u'opponent_abs_knockdowns_avg',
                    u'opponent_abs_pass_avg',
                    u'opponent_abs_reversals_avg',
                    u'opponent_abs_sub_attempts_avg',
                    u'opponent_abs_takedowns_landed_avg',
                    u'opponent_abs_takedowns_attempts_avg',
                    u'opponent_abs_sig_strikes_landed_avg',
                    u'opponent_abs_sig_strikes_attempts_avg',
                    u'opponent_abs_total_strikes_landed_avg',
                    u'opponent_abs_total_strikes_attempts_avg',
                    u'opponent_abs_head_strikes_landed_avg',
                    u'opponent_abs_head_strikes_attempts_avg',
                    u'opponent_abs_body_strikes_landed_avg',
                    u'opponent_abs_body_strikes_attempts_avg',
                    u'opponent_abs_leg_strikes_landed_avg',
                    u'opponent_abs_leg_strikes_attempts_avg',
                    u'opponent_abs_distance_strikes_landed_avg',
                    u'opponent_abs_distance_strikes_attempts_avg',
                    u'opponent_abs_clinch_strikes_landed_avg',
                    u'opponent_abs_clinch_strikes_attempts_avg',
                    u'opponent_abs_ground_strikes_landed_avg',
                    u'opponent_abs_ground_strikes_attempts_avg']

#forming a dataframe where columns are differences between fighter and opponent fight statistics
mrs=len(record_statistics)/2
record_diff_2_df=pd.DataFrame()
for i in range(mrs):
    record_diff_2_df[record_statistics[i]+'_diff_2']=ufc_fights_clean_stance[record_statistics[i]]-ufc_fights_clean_stance[record_statistics[i+mrs]]
    
#Lets start building a dataframe of all the stats which we know perform best
ufc_best_df=record_diff_2_df

#moving on to physical_stats:
physical_stats_diff_df=pd.DataFrame()
physical=[ [u'fighter_age',u'opponent_age'],[u'fighter_height',u'opponent_height'],[u'fighter_reach',u'opponent_reach']]
physical_stats_diff_df['age_diff']=ufc_fights_clean_stance['fighter_age'].apply(float)-ufc_fights_clean_stance['opponent_age'].apply(float)
physical_stats_diff_df['height_diff']=ufc_fights_clean_stance['fighter_height'].apply(float)-ufc_fights_clean_stance['opponent_height'].apply(float)
physical_stats_diff_df['reach_diff']=ufc_fights_clean_stance['fighter_reach'].apply(float)-ufc_fights_clean_stance['opponent_reach'].apply(float)
#ok so we keep ages absolute, but make differences for height and reach
ufc_best_df['fighter_age']=ufc_fights_clean_stance['fighter_age']
ufc_best_df['opponent_age']=ufc_fights_clean_stance['opponent_age']
ufc_best_df['height_diff']=physical_stats_diff_df['height_diff']
ufc_best_df['reach_diff']=physical_stats_diff_df['reach_diff']
#This includes all the best choices in terms of taking differences
for col in punch_statistics:
    ufc_best_df[col]=ufc_fights_clean_stance[col]
    
#Heres the score for our "best choices" (without stance included)
#THIS IS THE HIGHEST AVERAGE WE HAVE EVER GOTTEN, ITS A CROSS_VAL_SCORE of .71, 
#increases with cv (testing optimal value)
X=ufc_best_df.iloc[0:2101]
y=ufc_fights_clean_stance.iloc[0:2101]['result']
cross_val_score(model,X,y,cv=23).mean()

#adding stance to ufc_best_df
ufc_best_df['fighter_stance']=ufc_fights_clean_stance['fighter_stance']
ufc_best_df['opponent_stance']=ufc_fights_clean_stance['opponent_stance']

#Heres the score for our "best choices" (WITH stance included, increased by .005)
#THIS IS THE HIGHEST AVERAGE WE HAVE EVER GOTTEN, ITS A CROSS_VAL_SCORE of .71, 
#increases with cv (testing optimal value)
X=ufc_best_df.iloc[0:2101]
y=ufc_fights_clean_stance.iloc[0:2101]['result']
cross_val_score(model,X,y,cv=23).mean()

mlp=MLPClassifier(max_iter=1000,hidden_layer_sizes=(110,110),alpha=.00001,solver='sgd',random_state=3)
Xp=ufc_best_df.iloc[0:2101]
yp=ufc_fights_clean_stance.iloc[0:2101]['method']
mlp.fit(Xp,yp)

model=LogisticRegression()
X=ufc_best_df.iloc[0:2101]
y=ufc_fights_clean_stance.iloc[0:2101]['result']
model.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [19]:
ufc_predict('Petr Yan','Jose Aldo','July 12, 2020', 'July 12, 2020')

Petr Yan by KO/TKO


In [20]:
ufc_predict('Petr Yan','Jose Aldo','July 12, 2020', 'July 12, 2014')

Jose Aldo by U-DEC
